# Laboratory Exercise One - Writing Your Own Neural Network

## Predicting Species of the Iris Flower using the Famous Iris Dataset

In [1]:
## Import packages and dataset needed from Python
import numpy as np
import time
from sklearn import datasets
iris = datasets.load_iris()

In [2]:
iris.feature_names # These tell us what the inputs are, in this case measurements in cm of iris petals and sepals.

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [3]:
iris.target_names # These are the three species of iris flower that we are trying to predict from the inputs

array(['setosa', 'versicolor', 'virginica'],
      dtype='<U10')

### Prepare data - training and validation sets and class balancing

In [4]:
data = np.column_stack([iris.data,iris.target])

In [5]:
data_0 = data[0:40,:]
data_1 = data[50:90,:]
data_2 = data[100:140,:]
train = np.append(data_0,data_1,axis=0)
train = np.append(train,data_2, axis=0)

In [6]:
np.random.shuffle(train)

In [7]:
data_0_test = data[40:50,:]
data_1_test = data[90:100,:]
data_2_test = data[140:150,:]
test = np.append(data_0_test,data_1_test,axis=0)
test = np.append(test,data_2_test,axis=0)

In [8]:
np.random.shuffle(test)

### Set Up Neural Network Functions

In [9]:
def InitialiseNet(N_Neurons,Input,N_Outputs,Type):
    if Type == "Gaussian":
        W1 = np.random.normal(0,1,size=(N_Neurons, Input.shape[1] + 1))
        W2 = np.random.normal(0,1,size=(N_Outputs, N_Neurons + 1))
    elif Type == "Xavier":
        W1 = np.random.uniform(-np.sqrt(6/(Input.shape[1] + N_Neurons)),np.sqrt(6/(Input.shape[1] + N_Neurons)),size=(N_Neurons, Input.shape[1] + 1))
        W2 = np.random.uniform(-np.sqrt(6/(Input.shape[1] + N_Neurons)),np.sqrt(6/(Input.shape[1] + N_Neurons)),size=(N_Outputs, N_Neurons + 1))
    return W1, W2

In [10]:
def OneHot(lab):
    y = lab.astype(int)
    y_values = np.max(y) + 1
    y_hot = np.eye(y_values)[y]
    return y_hot

In [11]:
class Neuron:
    def Sigmoid(X):
        X = 1/(1 + np.exp(-X))
        return X
    
    def Tanh(X):
        X = np.tanh(X)
        return X
    
    def ReLU(X):
        X = X * (X > 0) + 0
        return X
        
    def Softmax(X):
        max_x = np.repeat(np.amax(X,axis=1),X.shape[1]).reshape(X.shape[0],X.shape[1])
        rowsum_x = np.repeat(np.sum(np.exp(X - max_x), axis=1),X.shape[1]).reshape(X.shape[0],X.shape[1])
        X = np.exp(X - max_x) / rowsum_x
        return X
    
    # Uncomment the code below and complete it for the advanced exercise
    
    '''
    def SoftPlus(X):
        X =
        return X
    '''

In [12]:
class NeuronPrime:
    def SigmoidPrime(X):
        X = Neuron.Sigmoid(X) * (1 - Neuron.Sigmoid(X))
        return X
    
    def TanhPrime(X):
        X = 1 - np.tanh(X)**2
        return X
    
    def ReLUPrime(X):
        X = X * (X > 0) + 0
        X[X>0] = 1
        return X
    
    # Uncomment the code below and complete it for the advanced exercise
    
    '''
    def SoftPlusPrime(X):
        X =
        return X
    '''

In [13]:
def Forwards(Input,Theta1,Theta2,Output,Activation,Regularisation):
    m = Input.shape[0]
    inp = np.column_stack([np.ones(m),Input])
    z = np.dot(inp,Theta1.T)
    a = np.column_stack([np.ones(m),Activation(z)])
    prds = Neuron.Softmax(np.dot(a,Theta2.T))
    if Regularisation:
        Reg = 0.0005/(2*m) * (np.sum(Theta1[:,1:]**2) + np.sum(Theta2[:,1:]**2))
        J = -1/m * np.sum(OneHot(Output)*np.log(prds)) + Reg
    else:
        J = -1/m * np.sum(OneHot(Output)*np.log(prds))
    return inp, z, a, prds, J

In [14]:
def Backwards(I,Z,A,Predicted,Output,Derivative,Regularisation):
    m = len(Predicted)
    D2 = Predicted - OneHot(Output)
    D1 = np.dot(D2,Theta2[:,1:])*Derivative(Z)
    Delta_2 = np.dot(D2.T,A)
    Delta_1 = np.dot(D1.T,I)
    if Regularisation:
        Delta_2 = Delta_2/m + np.column_stack([np.zeros(Theta2.shape[0]),(Theta2[:,1:]**2)*(0.0005/m)])
        Delta_1 = Delta_1/m + np.column_stack([np.zeros(Theta1.shape[0]),(Theta1[:,1:]**2)*(0.0005/m)])
    else:
        Delta_2 = Delta_2/m
        Delta_1 = Delta_1/m
    return Delta_2, Delta_1

In [15]:
def Accuracy(Pred,Labs):
    acc = np.sum(Labs == np.argmax(Pred, axis=1)).astype(int) / len(Pred)
    return acc

In [16]:
def NumericalGradients(x,y,initTh1,initTh2,Activation):
    e = 1e-5
    perturb_1 = np.zeros((initTh1.shape[0],initTh1.shape[1]))
    numgrad_1 = np.zeros((initTh1.shape[0],initTh1.shape[1]))
    perturb_2 = np.zeros((initTh2.shape[0],initTh2.shape[1]))
    numgrad_2 = np.zeros((initTh2.shape[0],initTh2.shape[1]))
    for i in range(initTh1.shape[0]):
        for j in range(initTh1.shape[1]):
            perturb_1[i,j] = e
            lg2 = initTh1 + perturb_1
            lg1 = initTh1 - perturb_1
            
            _,_,_,_, J_lg1 = Forwards(x,lg1,initTh2,y,Activation,Regularisation=False)
            
            _,_,_,_, J_lg2 = Forwards(x,lg2,initTh2,y,Activation,Regularisation=False)
             
            numgrad_1[i,j] = (J_lg2 - J_lg1) / (2*e)
            perturb_1[i,j] = 0
            
    for i in range(initTh2.shape[0]):
        for j in range(initTh2.shape[1]):
            perturb_2[i,j] = e
            lg2 = initTh2 + perturb_2
            lg1 = initTh2 - perturb_2
            
            _,_,_,_, J_lg1 = Forwards(x,initTh1,lg1,y,Activation,Regularisation=False)
            
            _,_,_,_, J_lg2 = Forwards(x,initTh1,lg2,y,Activation,Regularisation=False)
            
            numgrad_2[i,j] = (J_lg2 - J_lg1) / (2*e)
            perturb_2[i,j] = 0
    return numgrad_1, numgrad_2

## Numerical Gradient Checking

In [17]:
# Load saved theta weight values 
Theta1_init = np.load("Theta1_ini.npy")
Theta2_init = np.load("Theta2_ini.npy")

In [18]:
# Set theta values to initialise the weights
Theta1 = Theta1_init
Theta2 = Theta2_init

In [19]:
# Numerical Gradient Checking - are our analytical gradients working as intended?
numg1, numg2 = NumericalGradients(train[:,0:4],train[:,4],Theta1_init,Theta2_init,Neuron.Sigmoid)

In [20]:
numg1

array([[ 0.00214459,  0.01048171,  0.00728765,  0.00274818,  0.00034117],
       [ 0.04467811,  0.22744304,  0.15915843,  0.06726992,  0.01300343],
       [-0.0024653 , -0.00988403, -0.00894719,  0.00281946,  0.00230698],
       [-0.03018869, -0.17118835, -0.10431424, -0.08985582, -0.02771159],
       [ 0.01310075,  0.07659247,  0.04478484,  0.04573449,  0.01486346]])

In [21]:
numg2

array([[-0.19794978, -0.00333724, -0.05371027, -0.19131629, -0.14276425,
        -0.19249112],
       [ 0.22876753,  0.00205344,  0.08667067,  0.2226982 ,  0.16818173,
         0.21427808],
       [-0.03081775,  0.0012838 , -0.0329604 , -0.03138191, -0.02541748,
        -0.02178695]])

In [22]:
# One forward pass through the neural network to obtain the analytical gradients using initial theta weights.
# NOTE: Make sure that Regularisation is set to False! Otherwise the gradients will never match.
inp_numg, z_numg, a_numg, preds_numg, J_numg = Forwards(train[:,0:4],Theta1_init,Theta2_init,train[:,4],Neuron.Sigmoid,Regularisation=False)

In [23]:
# Backward pass for the gradients
d2, d1 = Backwards(inp_numg, z_numg, a_numg, preds_numg,train[:,4],NeuronPrime.SigmoidPrime,Regularisation=False)

In [24]:
d1

array([[ 0.00214459,  0.01048171,  0.00728765,  0.00274818,  0.00034117],
       [ 0.04467811,  0.22744304,  0.15915843,  0.06726992,  0.01300343],
       [-0.0024653 , -0.00988403, -0.00894719,  0.00281946,  0.00230698],
       [-0.03018869, -0.17118835, -0.10431424, -0.08985582, -0.02771159],
       [ 0.01310075,  0.07659247,  0.04478484,  0.04573449,  0.01486346]])

In [25]:
d2

array([[-0.19794978, -0.00333724, -0.05371027, -0.19131629, -0.14276425,
        -0.19249112],
       [ 0.22876753,  0.00205344,  0.08667067,  0.2226982 ,  0.16818173,
         0.21427808],
       [-0.03081775,  0.0012838 , -0.0329604 , -0.03138191, -0.02541748,
        -0.02178695]])

In [26]:
# Calculate the ratio of normed differences between Numerical and Analytical Gradients for each set of weights.
# The ratio should be less than 1e-7 if our analytical gradients are correct
np.linalg.norm(d1-numg1)/np.linalg.norm(d1+numg1)

7.6153832042909638e-11

In [27]:
np.linalg.norm(d2-numg2)/np.linalg.norm(d2+numg2)

3.1286655876792176e-11

## Training a Neural Network to Predict Species of Iris

In [57]:
#Randomly sample values to initialise theta weights
Theta1_init, Theta2_init = InitialiseNet(5,train[:,0:4],3,"Xavier")

In [58]:
# Set theta matrices to initial values
Theta1 = Theta1_init
Theta2 = Theta2_init

In [59]:
iters = 1000 # Numnber of training iterations. Because we are using the whole training set, iterations = epochs.

In [60]:
lr = 0.1 # The learning rate for Stochastic Gradient Descent.

In [61]:
start_time = time.time()
for i in range(iters):
    
    #Fowardpropagation
    inp, z, a, preds, J = Forwards(train[:,0:4],Theta1,Theta2,train[:,4],Neuron.ReLU,Regularisation=True)
    
    #Backpropagation
    d2, d1 = Backwards(inp, z, a, preds, train[:,4], NeuronPrime.ReLUPrime, Regularisation=True)
    
    #SGD weight update
    Theta1 = Theta1 - lr*d1
    Theta2 = Theta2 - lr*d2
    
    #Fowardpropagate through test dataset
    _,_,_,test_preds, J_test = Forwards(test[:,0:4],Theta1,Theta2,test[:,4],Neuron.ReLU,Regularisation=False)
    
    if (i + 1) % 100 == 0 or i == 0:
        print("Iteration :",i+1," Loss: ",J," Test Loss: ", J_test, "Accuracy (%): ", Accuracy(test_preds,test[:,4]))
    
end_time = time.time()
print("Total training time (seconds): ", end_time - start_time)

Iteration : 1  Loss:  3.23464719729  Test Loss:  2.73231282119 Accuracy (%):  0.333333333333
Iteration : 100  Loss:  0.361862877309  Test Loss:  0.347991444347 Accuracy (%):  0.8
Iteration : 200  Loss:  0.27382764989  Test Loss:  0.241857420213 Accuracy (%):  0.866666666667
Iteration : 300  Loss:  0.176516479245  Test Loss:  0.137513372196 Accuracy (%):  0.966666666667
Iteration : 400  Loss:  0.136927742682  Test Loss:  0.0912432410881 Accuracy (%):  0.966666666667
Iteration : 500  Loss:  0.118888349107  Test Loss:  0.0686860737721 Accuracy (%):  0.966666666667
Iteration : 600  Loss:  0.109018941589  Test Loss:  0.0556625731907 Accuracy (%):  1.0
Iteration : 700  Loss:  0.10279117196  Test Loss:  0.0471216999512 Accuracy (%):  1.0
Iteration : 800  Loss:  0.0982863439378  Test Loss:  0.0408951286172 Accuracy (%):  1.0
Iteration : 900  Loss:  0.0948036246598  Test Loss:  0.0361005556678 Accuracy (%):  1.0
Iteration : 1000  Loss:  0.0919917876254  Test Loss:  0.0322619729069 Accuracy (%):

# Your turn!

## The neural network below is broken. You need to fix the code to complete the exercise.

Once you have repaired your neural network, then train the following model:

1. Sigmoid neurons with regularisation.

If you have correctly implemented your neural network, then the training loss, test loss and test accuracy at 1000 iterations should be 0.166583658885, 0.156340344383 and 0.966666666667.

1. Tanh neurons with regularisation.
2. ReLU neurons with regularisation.
3. ReLU neurons with NO regularisation.

Train each model for 1000 iterations and record the training set loss, the test set loss, accuracy and training time. 
1. Which is the best model by test loss? 
2. Which model has the highest accuracy?
3. Of the two ReLU models, which has the lowest test loss? The model with regularisation or without it?
4. Which model was the fastest to train?

### If you finish early and want an additional challenge, train a neural network using the "softplus" neuron, which has the form f(x) = log(1 + e^x). Fill in the appropriate function and derivative in the classes above. HINT - the derivative of the softplus neuron is another kind of neuron we've used!

In [ ]:
# These are the initial theta values that you will use to intialise your models.

Theta1_init = np.load("Theta1_ini.npy")
Theta2_init = np.load("Theta2_ini.npy")

In [ ]:
# This passes the initial weight values to the theta weights your model will be training.

Theta1 = Theta1_init
Theta2 = Theta2_init

In [ ]:
# Number of training iterations

iters = 

In [ ]:
# Learning rate

lr = 0.01

In [ ]:
start_time = time.time()
for i in range(iters):
    
    #Fowardpropagation
    
    inp, z, a, preds, J = 
    
    #Backpropagation
    
    
    
    #SGD weight update
    Theta1 = Theta1 + lr*d1
    Theta2 = Theta2 - lr*d2
    
    #Fowardpropagate through test dataset
    _,_,_,test_preds, J_test = Forwards(test[:,0:4],Theta1,Theta2,test[:,4],Neuron.Tanh,False)
    
    if (i + 1) % 100 == 0 or i == 0:
        print("Iteration :",i+1," Loss: ",J," Test Loss: ", J_test, "Accuracy (%): ", Accuracy(test_preds,test[:,4]))

end_time = time.time()
print("Total training time (seconds): ", end_time - start_time)